# 0.0 - Imports

In [1]:
# database libraries
import psycopg2 as pg
import pandas as pd

import sys
from pathlib import Path

# ------ Importing database credentials package ------
# in jupyter (lab / notebook), based on notebook path
module_path = str(Path.cwd().parents[0] / "src" / "data")

if module_path not in sys.path:
    sys.path.append(module_path)

from config import config

# 1.0 - Database connection

In [2]:
# get access credentials
params = config()

# Connect to postgres DB
conn = pg.connect(**params)

# 2.0 - Data collection using pandas

In [3]:
# Execute a query
# you need to specify the schema name before the table: pa004 is the schema name.
query = """
        SELECT *
        FROM pa004.users u LEFT JOIN pa004.vehicle v ON (u.id = v.id)
                                LEFT JOIN pa004.insurance i ON (u.id = i.id)                                                                                     
        """

#query = 'SELECT * FROM information_schema.tables' # list all tables from database
# tables of interest: users, vehicle, insurance

# Retrieve query results
df = pd.read_sql_query(query, conn)

# Close connections
conn.close()

In [6]:
df.head()

,id,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,7,Male,23,11.0,152.0,1,< 1 Year,Yes,0,23367.0,249,0
1,13,Female,41,15.0,14.0,1,1-2 Year,No,1,31409.0,221,0
2,18,Female,25,35.0,152.0,1,< 1 Year,No,1,46622.0,299,0
3,31,Female,26,8.0,160.0,1,< 1 Year,No,0,2630.0,136,0
4,39,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0


In [5]:
# remove duplicated columns by NAME
# columns values were checked on the SQL query. 
# So its assured every value within the duplicated columns are the same on the same line.
# Eg: 'id' is '13' on row '1' for every column named 'id'
df = df.loc[:,~df.columns.duplicated()]

In [8]:
# saves to .csv file

path = str(Path.cwd().parents[0] / "data" / "raw")
filename_ = path + '/' + 'df_raw.csv'

df.to_csv(filename_)